# 범주형 데이터 처리: 원-핫 인코딩 (One-Hot Encoding)

이 노트북은 머신러닝 모델의 입력으로 사용하기 위해 **범주형(Categorical) 데이터**를 처리하는 방법을 다룹니다. 특히, 문자열로 된 범주형 데이터를 숫자 형태로 변환하는 **원-핫 인코딩** 기법을 `pandas`의 `get_dummies` 함수를 사용하여 알아봅니다.

### 왜 범주형 데이터를 변환해야 할까요?

머신러닝 알고리즘은 대부분 숫자 데이터를 기반으로 연산을 수행합니다. 따라서 '학생', '주부', '직장인'과 같은 문자열 데이터는 모델이 이해할 수 있는 숫자 형태로 변환해야 합니다.

단순히 '학생'=1, '주부'=2, '직장인'=3과 같이 숫자를 매핑(라벨 인코딩)하면, 모델은 이 숫자들 사이에 크기나 순서가 있다고 오해할 수 있습니다(예: 직장인(3)이 학생(1)보다 3배 더 중요하다). 

**원-핫 인코딩**은 이러한 문제를 해결하기 위해 각 범주를 새로운 특성(컬럼)으로 만들고, 해당 범주에 속하면 1, 아니면 0으로 표시하는 방법입니다.

### 1. 라이브러리 임포트

In [ ]:
import pandas as pd
import mglearn
import os
import numpy as np

### 2. 데이터 로드 및 선택

`mglearn` 라이브러리에 포함된 `adult` 인구 소득 데이터셋을 사용합니다. 분석에 사용할 일부 특성만 선택하여 데이터를 간소화합니다.

In [ ]:
# 데이터 파일 경로 설정
file_path = os.path.join(mglearn.datasets.DATA_PATH, "adult.data")

# 데이터 로드 (컬럼명 지정)
data_all = pd.read_csv(file_path, header=None, index_col=False, 
                   names=['age', 'workclass', 'fnlwgt', 'education',  'education-num',
                          'marital-status', 'occupation', 'relationship', 'race', 'gender',
                          'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
                          'income'])

# 분석에 사용할 특성 선택
data = data_all[['age', 'workclass', 'education', 'gender', 'hours-per-week', 'occupation', 'income']]

print("선택된 데이터 샘플 (처리 전):")
print(data.head())

### 3. 원-핫 인코딩 수행

`pandas`의 `get_dummies` 함수를 사용하여 데이터프레임 내의 모든 문자열(object) 및 범주형(category) 데이터를 원-핫 인코딩으로 자동 변환합니다.

- `workclass`, `education`, `gender`, `occupation`, `income` 컬럼이 숫자형 특성으로 변환됩니다.
- 예를 들어, `gender` 컬럼은 `gender_ Female`과 `gender_ Male`이라는 두 개의 새로운 컬럼으로 나뉩니다.

In [ ]:
data_dummies = pd.get_dummies(data)

print("원-핫 인코딩 후 데이터 샘플:")
print(data_dummies.head())

print("
생성된 컬럼 목록:")
print(data_dummies.columns)

### 4. 특성(X)과 타겟(y) 분리

원-핫 인코딩된 데이터에서 모델의 입력으로 사용될 특성(X)과 예측 대상인 타겟(y)을 분리합니다.

- **타겟(y)**: `income` 컬럼이 원-핫 인코딩되어 `income_ <=50K`와 `income_ >50K`로 나뉘었으므로, 이 중 `income_ >50K` (연 소득 5만 달러 초과)를 예측 목표로 설정합니다.

In [ ]:
# loc를 사용하여 컬럼 이름으로 데이터를 선택합니다.
X = data_dummies.loc[:, 'age':'occupation_ Transport-moving'] 
y = data_dummies.loc[:, 'income_ >50K']

print("--- 특성 데이터 (X) 샘플 ---")
print(X.head())

print("--- 타겟 데이터 (y) 샘플 ---")
print(y.head())

### 5. 모델 학습 및 평가

전처리된 데이터를 사용하여 로지스틱 회귀 모델을 학습시키고, 전체 데이터에 대한 정확도를 평가합니다.

**참고:** 실제 프로젝트에서는 모델의 일반화 성능을 정확히 평가하기 위해, 데이터를 훈련 세트와 테스트 세트로 분리(`train_test_split`)하여 평가해야 합니다. 여기서는 전처리 과정에 집중하기 위해 전체 데이터를 학습 및 평가에 사용했습니다.

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000) # 반복 횟수를 늘려 모델이 수렴하도록 돕습니다.
model.fit(X, y)

score = model.score(X, y)
print(f"모델 정확도: {score:.4f}")